In [ ]:
from os import listdir
from joblib import Parallel, delayed
import os, shutil
import numpy as np
import pandas as pd
import shlex, subprocess, sys
from PIL import Image
from os import path
import math
import cv2

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import ResNet152V2

In [ ]:
from Lib.MLG_lib import parse_lines_subplot,parse_line_subplot,parse_lines_text,parse_line_text_validate,parse_line_text,get_role,get_role_v 

In [ ]:
test_name = 'Open_Access'

# Compound Figure Detection

In [ ]:
target_dir = './open_access/' # the folder which contains images
test_dir = './data/pipeline_data/'+test_name+'/'
compound_dir = './data/' + test_name+'/compound/'
no_compound_dir = './data/' + test_name+'/no_compound/'
probability_dir = './data/' + test_name+'/prob/'

In [ ]:
os.mkdir(test_dir)
os.mkdir(compound_dir)
os.mkdir(no_compound_dir)
os.mkdir(probability_dir)

In [ ]:
compound_detector = keras.models.load_model('./data/weights/model_reproduce_final1')
resnet_model = ResNet152V2(weights="imagenet", pooling='max', include_top=False)

In [ ]:
def imgs_to_compound_chunk_directly(target_dir,file_list,comp_model,chunk):
    print('here')
    if len(file_list) > chunk:
        print('too many images')
        return
    x = []
    predictions = []
    procesed_file = []
    print(len(file_list))
    for file_name in file_list:
        try:
            img = load_img(target_dir+file_name,target_size=(224, 224))
            img = image.img_to_array(img)
            img = preprocess_input(np.expand_dims(img, axis=0))
            x.append(img)
            predictions.append(compound_detector.predict(img)[0])
            procesed_file.append(file_name)
        except:
            pass
        
    images = np.vstack(x)
    predictions = compound_detector.predict(images)
    return procesed_file, predictions

In [ ]:
def imgs_to_compound(img_dir,compound_dir,comp_model,test_dir,output,chunk):
    file_names = listdir(target_dir)
    file_list = []
    file_all = []
    counter = 0
    predictions = []
    first = 0
    rest = len(file_names)
    if len(file_names)>chunk:
        for file_name in file_names:
            file_list.append(file_name)
            counter += 1
            rest = rest -1
            if counter == chunk or rest==0:
                print(counter)
                file_list, predictions_1 = imgs_to_compound_chunk_directly(target_dir,file_list,comp_model,chunk)
                file_all.append(file_list)
                print('len_tmp: ', len(file_all))
                counter = 0
                file_list = []
                if first == 0:
                    predictions = predictions_1
                    first = 1
                else:
                    predictions = np.concatenate((predictions, predictions_1))
    else:
        print('less than chunk')
        for file_name in file_names:
            file_list.append(file_name)
        file_list, predictions = imgs_to_compound_chunk_directly(target_dir,file_list,comp_model,chunk)
        file_all.append(file_list)
        print('len_tmp2: ', len(file_all))
        
    f = open(test_dir+output, "w")
    print('file_len: ',len(file_all))
    print('prob_len: ',len(predictions))
    predicted_label = []
    probability_list = []
    for j,pred in enumerate(predictions):
        probability_list.append(pred[0])
        if pred[0] > 0.50:
            predicted_label.append('COMP')
        else:
            predicted_label.append('NOCOMP')

    f.close()
    def Extract(lst):
        result = []
        for i in range(len(lst)):
            for j in range(len(lst[i])):
                result.append(lst[i][j])
        return result
    
    file_all = Extract(file_all)
    print('file_len: ',len(file_all))
    print('label_len: ',len(predicted_label))
    print('prob_len: ', len(probability_list))
    my_dict = {'Img': [file[:-4] for file in file_all], 'Type_Prediction': predicted_label, 'Prob': probability_list}
    prediction = pd.DataFrame(my_dict)
    prediction.to_csv(probability_dir + 'ImageClef_Comp.csv', index = False)

In [ ]:
imgs_to_compound(target_dir,compound_dir,compound_detector,probability_dir,'ImageClef_Comp.txt',5000)

In [ ]:
# save file to compound directory and not compound directory
compound_df = pd.read_csv(probability_dir + 'ImageClef_Comp.csv')
for i in range(len(compound_df)):
    if compound_df['Type_Prediction'][i] == 'COMP':
        name = compound_df['Img'][i] + '.jpg'
        shutil.copy(target_dir+name, compound_dir+name)
    else:
        name = compound_df['Img'][i] + '.jpg'
        shutil.copy(target_dir+name, no_compound_dir+name)
# Return how many compound and nocompound images
compound_df['Type_Prediction'].value_counts()

In [ ]:
with open(probability_dir+'ImageClef_Comp.csv', 'r') as inp, open(probability_dir+'Comp.txt', 'w') as out:
    firstline = True
    for line in inp:
        if firstline:    #skip first line
            firstline = False
            continue
        line = line.replace(',', ' ')
        out.write(line)

# Subplot Detection with YoloV4

In [ ]:
folder_path = '/home/thuang12/Desktop/Misleading_Graph'
darknet_path = './darknet'
working_path = './darknet/'
voc_path = folder_path + '/data/data/voc_subplot_perf_orig_img.data'
cfg_path = folder_path + '/data/cfg/yolov4_subplot_orig_img_perf_valid.cfg'
weights_path = folder_path + '/data/yolov4_subplot_perf_orig_two_img_combined_and_all_original_4000.weights'
compound_list_path = folder_path + '/data/pipeline_data/'+test_name+'/compound_list.txt'
result_path = folder_path + '/data/pipeline_data/'+test_name+'/subplot_result1.txt'

In [ ]:
file_names = listdir(compound_dir)

In [ ]:
f = open(compound_list_path, "w")
for ff in file_names:
    f.write(folder_path+compound_dir[1:]+ff+'\n')
f.close()

In [ ]:
darknet_command = darknet_path+' detector test '+voc_path+' '+cfg_path+' '+weights_path+' '+ '-dont_show -ext_output < ' +compound_list_path+ ' > '+result_path 

In [ ]:
args = darknet_command
print(args)
p = subprocess.Popen(args,stdout=subprocess.PIPE,stderr=subprocess.PIPE,cwd=working_path, shell =True)
p.communicate()

In [ ]:
# get yolo result
f = open(result_path, "r")
content = f.readlines()
f.close()

In [ ]:
lines = parse_lines_subplot(content)

In [ ]:
f = open(probability_dir +'Subplot.txt', "w")
for line in lines:
    file_name = line[0].split('/')[-1]
    for i,prob in enumerate(line[2][1]):
        f.write(str(file_name[:-4])+'_'+str(i) +'.png' + "," + str(prob) + '\n')
f.close()

In [ ]:
Parallel(n_jobs=4)(delayed(parse_line_subplot)(lines[i],no_compound_dir) for i in range(len(lines)))  

# Image Classification

In [ ]:
bar_dir = './data/pipeline_data/'+test_name+'/Bar/'
line_dir = './data/pipeline_data/'+test_name+'/Line/'
heatmap_dir = './data/pipeline_data/'+test_name+'/Heatmap/'
scatter_dir = './data/pipeline_data/'+test_name+'/Scatter/'
violin_dir = './data/pipeline_data/'+test_name+'/Violin/'
box_dir = './data/pipeline_data/'+test_name+'/Box/'
area_dir = './data/pipeline_data/'+test_name+'/Area/'
map_dir = './data/pipeline_data/'+test_name+'/Map/'
table_dir = './data/pipeline_data/'+test_name+'/Table/'
radar_dir = './data/pipeline_data/'+test_name+'/Radar/'
pie_dir = './data/pipeline_data/'+test_name+'/Pie/'
venn_dir = './data/pipeline_data/'+test_name+'/Venn/'
pareto_dir = './data/pipeline_data/'+test_name+'/Pareto/'
diagnostic_dir = './data/pipeline_data/'+test_name+'/Diagnostic/'

In [ ]:
os.mkdir(bar_dir)
os.mkdir(line_dir)
os.mkdir(heatmap_dir)
os.mkdir(scatter_dir)
os.mkdir(violin_dir)
os.mkdir(box_dir)
os.mkdir(area_dir)
os.mkdir(map_dir)
os.mkdir(table_dir)
os.mkdir(radar_dir)
os.mkdir(pie_dir)
os.mkdir(venn_dir)
os.mkdir(pareto_dir)
os.mkdir(diagnostic_dir)

In [ ]:
img_classifier = keras.models.load_model('./data/weights/image_classfier_covid_top_nn.model')

In [ ]:
def imgs_to_type_chunck(target_dir,file_list,img_classifier,chunck):
    print('here')
    if len(file_list) > chunck:
        print('too many images')
        return
    
    x = []
    my_file = []
    
    for file_name in file_list:
        try:
            img = load_img(target_dir+file_name,target_size=(224, 224))
            img = image.img_to_array(img)
            #print(file_name)
            x.append(preprocess_input(np.expand_dims(img, axis=0)))
            my_file.append(file_name)
        except:
            continue
    
    images = np.vstack(x)
    features = resnet_model.predict(images)
    features = features.reshape((features.shape[0], 2048))
    
    predictions = img_classifier.predict(features)
    print(predictions[0])

    return my_file, predictions

In [ ]:
def imgs_to_type(no_compound_dir,img_classifier,test_dir,output,chunck):
    
    #half_file_number = round(len(listdir(no_compound_dir))/2)
    #file_names = listdir(no_compound_dir)[half_file_number:]
    file_names = listdir(no_compound_dir)
    
    file_list = []
    
    counter = 0
    predictions = []
    first = 0
    rest = len(file_names)
    
    predicted_file = []
    
    if len(file_names)>chunck:
        for file_name in file_names:
            file_list.append(file_name)
            
            counter += 1
            rest = rest -1
            if counter == chunck or rest==0:
                print(counter)
                files, predictions_1 = imgs_to_type_chunck(no_compound_dir,file_list,img_classifier,chunck)
                counter = 0
                file_list = []
                if first == 0:
                    predictions = predictions_1
                    first = 1
                else:
                    predictions = np.concatenate((predictions, predictions_1))
                
                predicted_file += files
                
    else:
        print('less than chunck')
        for file_name in file_names:
            try:
                img = load_img(no_compound_dir+file_name,target_size=(224, 224))
                file_list.append(file_name)
            except:
                file_names.remove(file_name)
                continue
        files, predictions = imgs_to_type_chunck(no_compound_dir,file_list,img_classifier,chunck)
        predicted_file += files
        
    print(len(predicted_file))
    f = open(probability_dir +'Bar.txt', "w")
    
    # "Diagnostic","BarGraph", "LineGraph",'Map','ParetoChart','PieChart','RadarPlot','ScatterGraph','Table','VennDiagram','AreaGraph','ViolinPlot','Heatmap','BoxPlot']
    for j,pred in enumerate(predictions):
        if pred[1] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], bar_dir+predicted_file[j])
            f.write(str(predicted_file[j]) + "," + str(pred[1]) + '\n')
        elif pred[10] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], area_dir+predicted_file[j])
        elif pred[7] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], scatter_dir+predicted_file[j])
        elif pred[12] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], heatmap_dir+predicted_file[j])
        elif pred[2] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], line_dir+predicted_file[j])
        elif pred[4] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], pareto_dir+predicted_file[j])
        elif pred[11] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], violin_dir+predicted_file[j])
        elif pred[0] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], diagnostic_dir+predicted_file[j])
        elif pred[3] > 0.90:
            shutil.copy(no_compound_dir+predicted_file[j], map_dir+predicted_file[j])
        elif pred[5] > 0.9:
            shutil.copy(no_compound_dir+predicted_file[j], pie_dir+predicted_file[j])
        elif pred[6] > 0.9:
            shutil.copy(no_compound_dir+predicted_file[j], radar_dir+predicted_file[j])
        elif pred[8] > 0.9:
            shutil.copy(no_compound_dir+predicted_file[j], table_dir+predicted_file[j])
        elif pred[9] > 0.9:
            shutil.copy(no_compound_dir+predicted_file[j], venn_dir+predicted_file[j])
        elif pred[13] > 0.9:
            shutil.copy(no_compound_dir+predicted_file[j], box_dir+predicted_file[j])
        
    f.close()

In [ ]:
imgs_to_type(no_compound_dir,img_classifier,bar_dir,'Bar_Prob.txt', 3500)

<hr style="border:2px solid black"> </hr>


In [ ]:
os.mkdir('./' + test_name + '_swt/')

# Stroke-Width-Transform 

<hr style="border:2px solid black"> </hr>


# Text Localization with YoloV4

In [ ]:
folder_path = '/home/thuang12/Desktop/Misleading_Graph'
target_dir = './OA_swt_AO/'
working_path = './darknet'
darknet_path = './darknet'
voc_path = folder_path + '/data/data/voc_text_swt.data'
cfg_path = folder_path + '/data/cfg/yolov4_text_swt_valid.cfg'
weights_path = folder_path + '/data/yolov4_text_swt_last.weights'
file_list_path = folder_path + '/data/pipeline_data/'+test_name+'/img_list.txt'
result_path = folder_path + '/data/pipeline_data/'+test_name+'/text_result.txt'
output_dir = folder_path + '/data/pipeline_data/'+test_name+'/text_localization/'

In [ ]:
file_names = listdir(target_dir)

In [ ]:
f = open(file_list_path, "w")
for ff in file_names:
    if ff[-4:] in ['.png', '.jpg']:
        f.write(folder_path+target_dir[1:]+ff+'\n')
f.close()

In [ ]:
darknet_command = darknet_path+' detector test '+voc_path+' '+cfg_path+' '+weights_path+' '+ '-dont_show -ext_output < ' +file_list_path+ ' > '+result_path

In [ ]:
args = darknet_command
p = subprocess.Popen(args,stdout=subprocess.PIPE,stderr=subprocess.PIPE,cwd=working_path, shell = True)
p.communicate()

In [ ]:
f = open(result_path, "r")
content = f.readlines()
f.close()
len(content)

In [ ]:
labeled_data = './OA_swt_AO/'
data_path = './OA_swt_AO/'
org_img_dir = './data/pipeline_data/'+test_name+'/Bar/'

In [ ]:
lines = parse_lines_text(content)

In [ ]:
file_names = listdir(org_img_dir)
file_names = [file for file in file_names if file[-4:] in ['.png', '.jpg']]

In [ ]:
# Clean up the tmp folder
tmp_list = listdir('./data/tmp')
for i in range(len(tmp_list)):
    shutil.rmtree('./data/tmp/' + tmp_list[i])

In [ ]:
Parallel(n_jobs=4)(delayed(parse_line_text_validate)(lines[i],org_img_dir,labeled_data) for i in range(len(lines)))

In [ ]:
# Change all bar charts type from jpg to png
all_img = listdir(labeled_data)
for i in range(len(all_img)):
    if all_img[i][-4:] == '.jpg':
        im = Image.open(target_dir + all_img[i])
        #im.save(target_dir + all_img[i][:-4] + '.png')
        im.convert('RGB').save(target_dir + all_img[i][:-4] +'.png', "PNG", optimize=True)
        os.remove(target_dir + all_img[i])
        im.close()

<hr style="border:2px solid black"> </hr>

## Run get role in rev environment 
- under rev folder
- conda activate rev
- python run_get_role.py

<hr style="border:2px solid black"> </hr>

# Data pre-processing and feature engineering

In [ ]:
def pre_processing(labeled_data_path, img_name, save_path):
    for i in range(len(img_name)):
        file_name = img_name[i][:-4]
        #if path.exists(labeled_data_path+file_name+'-texts-all.csv'):
        try:
            df = pd.read_csv(labeled_data_path+file_name+'-texts-all.csv')
            # get y label 
            y_label = df[df['type']=='y-axis-label'].value_counts('x')
            y_label = y_label.iloc[:1]
            if len(y_label) != 0:
                y_label = y_label.index.tolist()[0][0]
                width = float(np.mean(df[(df['type']=='y-axis-label') & (df['x'] == y_label)]['width']))
                for j in range(len(df)):
                    value = df['x'][j] + df['width'][j]
                    value1 = df['x'][j]
                    if df['type'][j] == 'y-axis-label' and (value < y_label + width - 6 or value > y_label + width + 6):
                        df.at[j, 'type'] = 'y-axis-title'
                    #if value > y_label+width-6 and value < y_label+width+6 and df['type'][j] != 'y-axis-label':
                    if value1 >= y_label -3 and value1 <= y_label+width and df['type'][j] != 'y-axis-label':
                        df.at[j, 'type'] = 'y-axis-label'
                    #print('v:', value, 'y:', y_label+width)
            # get x label
            x_label = df[df['type']=='x-axis-label'].value_counts('y')
            x_label = x_label.iloc[:1]

            if len(x_label) != 0:
                x_label = x_label.index.tolist()[0][0]
                height = float(np.mean(df[(df['type']=='x-axis-label') & (df['y'] == x_label)]['height']))
                for j in range(len(df)):
                    value = df['y'][j]+height
                    if df['type'][j] == 'x-axis-label' and (value < x_label+height-4 or value > x_label+height+4):
                        df.at[j, 'type'] = 'x-axis-title'
                    if value > x_label+height-4 and value < x_label+height+4 and df['type'][j] != 'x-axis-label':
                        df.at[j, 'type'] = 'x-axis-label'

            df.to_csv(save_path+file_name+'-texts-all.csv', index=False)
            print(file_name, ' finished', )

        except:
            #df.to_csv(save_path+file_name+'-texts-all.csv', index=False)
            pass

In [ ]:
labeled_data = './OA_swt_AO/'
file_names = listdir(labeled_data)
bar_img = [ff for ff in file_names if ff[-4:] in ['.png', '.jpg']]
save_path = './OA_swtAO_processed/'
os.mkdir(save_path)
pre_processing(labeled_data, bar_img, save_path)

In [ ]:
# save images to folder where processed csv files are
no_process_path = './OA_swt_AO/'
files = listdir(no_process_path)
for i in range(len(files)):
    if files[i][-4:] == '.png':
        shutil.copy(no_process_path+files[i], './OA_swtAO_processed/'+files[i])

In [ ]:
def get_prob_dic(probability_dir):
    #print(probability_dir)
    #f1 = open(probability_dir +'Comp.txt', "r")
    #lines_1 = f1.readlines()
    #f1.close()
    #f2 = open(probability_dir +'Subplot.txt', "r")
    #lines_2 = f2.readlines()
    #f2.close()
    #f3 = open(probability_dir +'3D.txt', "r")
    #lines_3 = f3.readlines()
    #f3.close()
    f4 = open(probability_dir +'Bar.txt', "r")
    lines_4 = f4.readlines()
    f4.close()
    """
    comp_dic = {}
    
    for line in lines_1:
        comp_dic[line.split(' ')[0]] = line.split(' ')[2][:-1]
    
    subp_dic = {}
    
    for line in lines_2:
        subp_dic[line.split(',')[0]] = line.split(',')[1][:-1]
    
    d3_dic = {}
    
    #for line in lines_3:
    #    d3_dic[line.split(',')[0]] = line.split(',')[1][:-1]
    """
    
    bar_dic = {}
    
    for line in lines_4:
        bar_dic[line.split(',')[0].replace('.jpg', '.png')] = line.split(',')[1][:-1]
        #bar_dic[line.split(',')[0]] = line.split(',')[1][:-1]
    
    #print(comp_dic)
    
    #return comp_dic,subp_dic,d3_dic,bar_dic
    return bar_dic

In [ ]:
def get_prob_file(file,bar_dic):
    #comp_prob = -1
    #subp_prob = -1
    #d3_prob = -1
    bar_prob = -1
    #if '_' in file:
    #    comp_prob = comp_dic[file.split('_')[0]+'.png']
    """
    if file[-7] == '_':
        comp_prob = comp_dic[file[:-7]]
        subp_prob = subp_dic[file]
    elif file[-6] == '_':
        comp_prob = comp_dic[file[:-6]]
        subp_prob = subp_dic[file]
    else:
        comp_prob = comp_dic[file[:-4]]
        subp_prob = -1
    """    
    #d3_prob = d3_dic[file]
    bar_prob = bar_dic[file]
    
    #print(d3_prob)
    
    #return comp_prob,subp_prob,d3_prob,bar_prob
    return bar_prob

In [ ]:
def check_digit(word):
    word = str(word)
    word = word.replace(',','')
    word = word.replace('-','')
    word = word.replace('|','')
    try:
        float(word)
        return True
    except:
        return False

In [ ]:
def process_digit(word):
    word = str(word)
    word = word.replace(',','')
    word = word.replace('-','')
    word = word.replace('|','')
    
    return float(word)

In [ ]:
def keep_clear_files(file,labeled_data):
    
    try:
        tb = pd.read_csv(labeled_data + file[:-4] + '-texts-all.csv')
    except:
        return 0
        
    y_axis_value = tb[tb['type'] == 'y-axis-label']['text'].tolist()
    
    if len(y_axis_value)<4:
        return 0
    #x_axis_pos_y = tb[tb['type'] == 'x-axis-label']['y'].tolist()
    
    #if len(x_axis_pos_y)<2:
    #    return 0
    
    return 1

In [ ]:
# Manually labeded data
#misleading_label_dir = '/home/thuang12/Desktop/Misleading/'
#other_label_dir = './data/pipeline_data/Open_Access/Bar'

In [ ]:
import json
f = open('./OA_label/oa2_label.json',)
data = json.load(f)
f.close()

In [ ]:
image_name  = []
image_label = []
for i in range(len(data)):
    tmp_name = data[i]['image'].split('/')[-1][:-12]
    image_name.append(tmp_name)
    image_label.append(data[i].get('choice'))

In [ ]:
misleading_label_dir = []
other_label_dir = []
for i in range(len(image_name)):
    if image_label[i] == 'misleading':
        misleading_label_dir.append(image_name[i])
    else:
        other_label_dir.append(image_name[i])

In [ ]:
def feature_extract_training(labeled_data, file,bar_dic):
    y = -1
    #if file in listdir(misleading_label_dir):
    #    y = 1
    #elif file in listdir(other_label_dir):
    #    y = 0
    #else:
    #    return 
    
    if file[:-4] in misleading_label_dir:
        y = 1
    elif file[:-4] in other_label_dir:
        y = 0
    else:
        y = 0
           
            
    im = cv2.imread(labeled_data+file)
    
    h,w,c = im.shape
    
    try:
        tb = pd.read_csv(labeled_data + file[:-4] + '-texts-all.csv')
    except:
        return
    #print(file)
    #comp_prob,subp_prob,d3_prob,bar_prob = get_prob_file(file,comp_dic,subp_dic,d3_dic,bar_dic)
    bar_prob = get_prob_file(file,bar_dic)
    #print(d3_prob)

    y_axis_value = tb[tb['type'] == 'y-axis-label']['text'].tolist()
    y_axis_font = tb[tb['type'] == 'y-axis-label']['font'].tolist()
    y_axis_prob = tb[tb['type'] == 'y-axis-label']['prob'].tolist()
    y_axis_conf = tb[tb['type'] == 'y-axis-label']['conf'].tolist()

    y_axis_pos_y = tb[tb['type'] == 'y-axis-label']['y'].tolist()
    y_axis_pos_y = [float(yy) for index, yy in enumerate(y_axis_pos_y) if check_digit(y_axis_value[index])]

    y_axis_value = [process_digit(yy) for yy in y_axis_value if check_digit(yy)]

    y_axis_pairs = zip(y_axis_pos_y, y_axis_value)
    res = sorted(y_axis_pairs, key=lambda x: x[0], reverse=True)

    y_axis_pos_y = sorted(y_axis_pos_y)
    y_axis_value = sorted(y_axis_value)
    
    steps = []
    for index, yy in enumerate(y_axis_pos_y):
        if index == len(y_axis_pos_y) - 1:
            break
        step = y_axis_pos_y[index + 1] - yy
        steps.append(step)
    
    #if file == 'PMC3144924_pone.0022627.g003_2.png':
    #    print(steps)
    
    if len(steps) >= 1:
        average_step = np.mean(steps)
    else:
        average_step = -1
        
    #print(average_step)
    increase_rates = []
    decimal = 0
    for index, yy in enumerate(y_axis_value):           
        if index < len(y_axis_value) -1 and yy.is_integer() != y_axis_value[index+1].is_integer():
            decimal += 1
            
    for index, yy in enumerate(y_axis_value):
        if index == len(y_axis_value) - 1:
            break
        try:
            increase_rate = (y_axis_value[index + 1] - yy) / steps[index]
            increase_rates.append(increase_rate)
        except:
            continue
            
    increase_rate_std = np.std(increase_rates/np.mean(increase_rates))
    
    #common_rates,rates_count = common_rough(increase_rates)
    #if len(common_rates) == 1:
    #    increase_rate_std = 0
    #else:
    #    increase_rate_std = np.std(common_rates)
    
    
    #if decimal >= 2:
    #    average_increase_rate = 0
    #else:
    average_increase_rate = np.mean(increase_rates)

    x_axis_pos_y = tb[tb['type'] == 'x-axis-label']['y'].tolist()
    x_axis_pos_y = [float(xx) for xx in x_axis_pos_y]

    average_x_axis_pos_y = np.mean(x_axis_pos_y)

    if np.std(x_axis_pos_y) > 2:
        # print("double x axes")
        x_axis_pos_y = [float(xx) for xx in x_axis_pos_y if xx > average_x_axis_pos_y]
    average_x_axis_pos_y = np.mean(x_axis_pos_y)

    inference_flag_1 = 0
    min_y_1 = 0
    # print(res)
    # print(average_step)
    if len(res) <= 3:
        return []
    
    #if abs(res[0][0] - average_x_axis_pos_y) > abs(average_step / 2) and check_digit(res[1][1]) and res[-1][0] < average_x_axis_pos_y:
        # inference
        # print('inference one')
    #    inference_flag_1 = 1
        # print(res[0][1])
        # print(average_increase_rate)
        # print(abs(res[0][0] - average_x_axis_pos_y))
    #    min_y_1 = res[0][1] - average_increase_rate * (res[0][0] - average_x_axis_pos_y)

    ##### try to find the bar in the plot #####
    #if res[0][1] != 0:
    y_ratio = res[0][0] / h
    if min(y_axis_value)!=0 and y_ratio<0.85:
        min_y_1 = res[0][1] - average_increase_rate * average_step    
        inference_flag_1 = 1
        #if file == 'PMC3144924_pone.0022627.g003_2.png':
        #    print('res: ', res[0][1], average_increase_rate, average_step)
        #    print('min_y: ', min_y_1)
    #inference_flag_2 = 0
    #if not check_digit(res[0][1]) and check_digit(res[1][1]) and inference_flag_1 == 0:
        # inference
        # print('inference two')
    #    inference_flag_2 = 1
    #    min_y_1 = res[1][1] - average_increase_rate * (res[1][0] - x_axis_pos_y)

    #increase_rate_std = np.std(increase_rates)
    inference_flag = 0
    if inference_flag_1 == 0: #and inference_flag_2 == 0:
        #min_y = res[0][1]/(max(y_axis_value)+1)
        min_y = min(y_axis_value)
    else:
        inference_flag = 1
        #print('assign')
        #print(inference_flag_1)
        #print(min_y_1)
        #min_y = min_y_1/(max(y_axis_value)+1)
        #min_y = min(y_axis_value)
        min_y = min_y_1
        
    if math.isnan(min_y):
        #print(len(res))
        min_y = 0

    if np.isnan(increase_rate_std):
        increase_rate_std = 0

    if np.isnan(average_x_axis_pos_y):
        average_x_axis_pos_y = -1
    
    #(res[0][0] - average_x_axis_pos_y)
    #return [y, [inference_flag ,(res[0][0] - average_x_axis_pos_y),  min_y, increase_rate_std,np.mean(y_axis_font),np.mean(y_axis_prob),np.mean(y_axis_conf), float(comp_prob),float(subp_prob),float(bar_prob)]]    
    #return [y, [[y ,file],inference_flag,min_y,decimal,np.mean(y_axis_prob),np.mean(y_axis_conf),float(bar_prob),increase_rate_std]]
    return [y, [inference_flag,min_y,decimal,np.mean(y_axis_prob),np.mean(y_axis_conf),float(bar_prob),increase_rate_std]]

In [ ]:
def get_features_trainining(img_list,labeled_data,bar_dic):
    X = []
    y = []
    files = []
    for i in range(len(img_list)):
        if keep_clear_files(img_list[i],labeled_data):
            #res = feature_extract_training(labeled_data,img_list[i],comp_dic,subp_dic,d3_dic,bar_dic)
            res = feature_extract_training(labeled_data,img_list[i],bar_dic)
            if res != None and len(res) == 2 and len(res[1])==7:
                X.append(res[1])
                y.append(res[0])
                files.append(img_list[i])

    return X,y,files

In [ ]:
#labeled_data = './OA_swt2_processed/'
#probability_dir = './data/pipeline_data/'+test_name+'/prob/'
labeled_data = '/home/thuang12/Desktop/FN_goodtext/'
probability_dir = './data/pipeline_data/Open_Access2/prob/'
file_names = listdir(labeled_data)
bar_img = [ff for ff in file_names if ff[-4:] in ['.png', '.jpg']]

In [ ]:
#comp_dic,subp_dic,d3_dic,bar_dic = get_prob_dic(probability_dir)
bar_dic = get_prob_dic(probability_dir)
#X,y,files = get_features_trainining(bar_img,labeled_data,comp_dic,subp_dic,d3_dic,bar_dic)
X,y,files = get_features_trainining(bar_img,labeled_data,bar_dic)

# Graphical Integrity Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
graphical_integrity_classification = RandomForestClassifier(max_depth=7, random_state=1, n_estimators = 500, oob_score=True, n_jobs=-1)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.2, random_state=30)

In [ ]:
graphical_integrity_classification.fit(train_X, train_y)

In [ ]:
pred_y = graphical_integrity_classification.predict(test_X)